In [1]:
import tensorflow as tf

2024-10-10 00:33:54.475542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 00:33:54.490920: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 00:33:54.494980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 00:33:54.511384: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 00:33:56.296294: W tensorflow/compiler/tf2tens

In [2]:
print(tf.__version__)

2.17.0


In [4]:
from tensorflow.keras.callbacks import TensorBoard # this is for showing the graphs in live
import datetime

In [5]:
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

This is the path to my data

In [6]:
train_dir = 'data/train'
valid_dir = 'data/valid'
test_dir = 'data/test'

Loading the metadata

In [7]:
train_csv = pd.read_csv(os.path.join(train_dir, '_annotations.csv'))
valid_csv = pd.read_csv(os.path.join(valid_dir, '_annotations.csv'))
test_csv = pd.read_csv(os.path.join(test_dir, '_annotations.csv'))

Create the data generators

In [8]:
datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_csv.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,0674_jpg.rf.e02a155a0c135687b9301ff9a20d220a.jpg,512,256,Implant,175,116,206,153
1,0674_jpg.rf.e02a155a0c135687b9301ff9a20d220a.jpg,512,256,Fillings,170,109,189,133
2,0674_jpg.rf.e02a155a0c135687b9301ff9a20d220a.jpg,512,256,Implant,221,124,257,178
3,0674_jpg.rf.e02a155a0c135687b9301ff9a20d220a.jpg,512,256,Implant,302,126,329,175
4,0674_jpg.rf.e02a155a0c135687b9301ff9a20d220a.jpg,512,256,Implant,335,114,360,154


In [9]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory=train_dir,
    x_col='filename',
    y_col='class',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Cambiado a 'categorical' para múltiples clases
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=valid_csv,
    directory=valid_dir,
    x_col='filename',
    y_col='class',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Cambiado a 'categorical' para múltiples clases
)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_csv,
    directory=test_dir,
    x_col='filename',
    y_col=None,
    target_size=(150, 150),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

Found 8030 validated image filenames belonging to 4 classes.
Found 780 validated image filenames belonging to 4 classes.
Found 473 validated image filenames.


Showing the images

In [ ]:


def show_images(generator):
    x_batch, y_batch = next(generator)
    for i in range(0, 9):                
        plt.subplot(330 + 1 + i)
        plt.imshow(x_batch[i])
        plt.title(f'Clase: {y_batch[i].argmax()}')  # Mostrar la clase de la imagen
        plt.axis('off')
    plt.show()    
    

show_images(train_generator)

In [ ]:
show_images(valid_generator)

Define models

In [ ]:
def create_simple_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(150, 150, 3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def create_cnn_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def create_rnn_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), input_shape=(None, 150, 150, 3)),
        tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2))),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
        tf.keras.layers.LSTM(128),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Configurate TensorBoard for showing the learning

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training models using TensorBoard:

In [ ]:
simple_model = create_simple_model()
cnn_model = create_cnn_model()
rnn_model = create_rnn_model()


history_simple = simple_model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=[tensorboard_callback])
history_cnn = cnn_model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=[tensorboard_callback])
history_rnn = rnn_model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=[tensorboard_callback])

# Showing results

In [ ]:
# Mostrar los gráficos de aprendizaje:

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
def plot_learning_curves(histories, titles):
    plt.figure(figsize=(15, 5))
    for i, history in enumerate(histories):
        plt.subplot(1, 3, i+1)
        plt.plot(history.history['accuracy'], label='train_accuracy')
        plt.plot(history.history['val_accuracy'], label='val_accuracy')
        plt.title(titles[i])
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
    plt.show()

plot_learning_curves([history_simple, history_cnn], ['Simple Model', 'CNN Model'])